### Rank vs Dense Rank

In [0]:
SELECT s.store_name,
  ROUND(SUM(f.total_amount), 2) AS total_sales,
  RANK() OVER (ORDER BY SUM(f.total_amount) DESC) AS rank,
  DENSE_RANK() OVER (ORDER BY SUM(f.total_amount) DESC) AS dense_rank


FROM learning_pradeep.pipeline.silver_fact_sales f
LEFT JOIN learning_pradeep.pipeline.silver_dim_stores s ON f.store_sk = s.store_sk

GROUP BY s.store_name

### Top N per Store

In [0]:
select * from 
(select s.store_name , p.product_name , SUM(f.total_amount) as total,
row_number() over(partition by s.store_name order by SUM(f.total_amount) desc) as top

from learning_pradeep.pipeline.silver_fact_sales f 
left join learning_pradeep.pipeline.silver_dim_stores s on f.store_sk = s.store_sk
left join learning_pradeep.pipeline.silver_dim_products p on f.product_sk = p.product_sk

group by s.store_name, p.product_name ) 

where top <= 3 
limit 20

### MoM Growth

In [0]:
SELECT d.year, d.month,
SUM(f.total_amount) revenue,
LAG(SUM(f.total_amount)) OVER(ORDER BY d.year, d.month) AS prev_month,
(SUM(f.total_amount)-LAG(SUM(f.total_amount)) OVER(ORDER BY d.year, d.month))
       / LAG(SUM(f.total_amount)) OVER(ORDER BY d.year, d.month) * 100 AS mom_growth


FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f 
LEFT JOIN learning_pradeep.pipeline.silver_dim_dates d ON f.sales_date = d.full_date

GROUP BY d.year, d.month

In [0]:
SELECT d.year, d.month,
SUM(f.total_amount) revenue,
AVG(SUM(f.total_amount)) OVER(ORDER BY d.year, d.month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ROLLING_AVG 


FROM learning_pradeep.pipeline.bronze_fact_sales_normalized f 
LEFT JOIN learning_pradeep.pipeline.silver_dim_dates d ON f.sales_date = d.full_date

GROUP BY d.year, d.month